In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet50
from collections import OrderedDict

from pooling import SPoC, MAC, GeM

In [10]:
class CMG(nn.Module):
    def __init__(self,freeze=False,
                num_classes=20):
        super().backbone = resnet50(pretrained=True)
        del self.backbone.avgpool, self.backbone.fc
        
        if freeze :
            for param in self.backbone.parameters():
                param.requires_grad = False
                
        poolings = [GeM(), SPoC(), MAC()]
        
        for idx, module in enumerate(pooling):
            setattr(self, f'pool-{idx}', module)
            setattr(self, f'fc-{idx}', nn.Linear(2048,512))
            
        self.flatten =  nn.Flatten()
        
#         self.bn = nn.BatchNorm1d(2048)
#         self.relu = nn.ReLU()
#         self.clf = nn.Linear(2048,num_classes)
        
        self.classification = nn.Sequential(
            OrderedDict({
                'bn': nn.BatchNorm1d(2048),
                'relu':nn.ReLU(),
                'clf': nn.Linear(2048,num_classes)
            })
        )
    def forward(self,x):
        for _, module in self.backbone.named_children():
            x = module(x)
        
    global_description = []
    for idx in range(3):
            pool = getattr(self, f'pool-{idx}')
            fc = getattr(self, f'fc-{idx}')
            
            if idx == 0:
                y_hat = self.classification(tmp_x)
            
            tmp_x = pool(x)
            tmp_x = self.flatten(tmp_x)
            tmp_x = fc(tmp_x)
            tmp_x = F.normalize(tmp_x)
            global_description.append(tmp_x)
            
        # Concat
    x = torch.cat(global_description, 1)
    x = F.normalize(x)
    return x, y_hat


IndentationError: unexpected indent (<ipython-input-10-c106674eaded>, line 35)